In [1]:
import os
import json
from copy import copy
import numpy
import torch

In [2]:
folder = 'exp/json'
files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
for f in files:
    f = open(os.path.join(folder, f))
    recording = json.loads(f.readline())
    # Removing multiple-speaker segments
    recording['segments'] = [segment for segment in recording['segments'] if len(segment['speakers']) == 1]
    # Removing extra speakers
    recording['segments'] = [segment for segment in recording['segments'] if segment['speakers'][0]['speaker_id'] in ['A', 'B']]
    # Dealing with multiple-vector segments
    new_segments = []
    for segment in recording['segments']:
        for i in range(len(segment['ivectors'])):
            new_segment = copy(segment)
            new_segment['ivectors'] = [segment['ivectors'][i]]
            new_segment['xvectors'] = [segment['xvectors'][i]]
            new_segments.append(new_segment)
    recording['segments'] = new_segments
    # Lists to numpy arrays
    for segment in recording['segments']:
        segment['ivectors'][0]['value'] = numpy.asarray(segment['ivectors'][0]['value'])
        segment['xvectors'][0]['value'] = numpy.asarray(segment['xvectors'][0]['value'])
    # Getting speakers list
    speakers = list(set([segment['speakers'][0]['speaker_id'] for segment in recording['segments']]))
    # Getting speakers oracle models
    oracle_limit = 5
    oracle_segments = [[segment for segment in recording['segments'] if segment['speakers'][0]['speaker_id'] == speaker][:oracle_limit] for speaker in speakers]
    oracle_models = []
    for n, speaker_n in enumerate(speakers):
        ivector_n = oracle_segments[n][0]['ivectors'][0]['value']
        xvector_n = oracle_segments[n][0]['xvectors'][0]['value']
        for segment in oracle_segments[n][1:]:
            ivector_n = ivector_n + segment['ivectors'][0]['value']
            xvector_n = xvector_n + segment['xvectors'][0]['value']
        ivector_n = ivector_n / len(oracle_segments[n])
        xvector_n = xvector_n / len(oracle_segments[n])
        oracle_models.append({ 'speaker_id': speaker_n, 'ivector': ivector_n, 'xvector': xvector_n })